## Cross-Bucket Tests on all 3 Fragment Types using Kalman Filter

### Part A: (1) Import Fragment Data from MATLAB, (2) Save Them in a .pickle file, and (3) Load Them In

###### (HOWEVER, YOU DON'T NEED TO LOAD THEM IN AGAIN SINCE STEP 1 ALREADY DOES THAT FOR YOU!)

In [218]:
## (Configuration) Allows you to return multiple variables from a single cell ##
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Allows you to import files from another folder in current directory ## 
import os 
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import standard package ###
import numpy as np
from scipy import io
import sys

# Specify Fragment Types to be used for this Anaylsis 
frag_type = ['AD', 'HV', 'VM'] # ['AccDec', 'HillValley', 'VelMin']

In [219]:
 ## (OLD) Import Data ## 
# We'll load in position data and derive velocity and acceleration from it 

folder = '/Users/rbhatt6/Documents/MATLAB/' # For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')
   
    outputX = "sortX_"+frag_type[i]
    locals()[outputX] = io.loadmat(folder + outputX + '.mat')

    outputY = "sortY_"+frag_type[i]
    locals()[outputY] = io.loadmat(folder + outputY + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[outputX] = np.squeeze(list(locals()[outputX].values())[3])
    locals()[outputY] = np.squeeze(list(locals()[outputY].values())[3])

In [15]:
## (OLD) Save Data in .pickle files ## 
import pickle

data_folder='/Users/rbhatt6/Documents/MATLAB/' # Folder where you want to save the data

for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    outputX = "sortX_"+frag_type[i]
    outputY = "sortY_"+frag_type[i]

    with open(data_folder + input + '.pickle','wb') as f:
        pickle.dump(locals()[input],f)

    with open(data_folder + outputX + '.pickle','wb') as f:
        pickle.dump(locals()[outputX],f)

    with open(data_folder + outputY + '.pickle','wb') as f:
        pickle.dump(locals()[outputY],f)

In [228]:
## (UPDATED) Import Data ##
# We'll load in position data and derive velocity and acceleration from it 

folder = '/Users/rbhatt6/Documents/MATLAB/' # For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')

    output = "sortOut_" + frag_type[i]
    locals()[output] = io.loadmat(folder + output + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[output] = np.squeeze(list(locals()[output].values())[3])

In [229]:
sortOut_AD[0][0].shape[0]
float(sortOut_AD[0][0][0])
test1 = float(sortOut_AD[0][0][0])
test2 = float(sortOut_AD[0][1][0])
test3 = [test1, test2]
test3

sortOut_AD[0][5][0]

sortOut_AD[0][0]

np.nditer(sortOut_AD[0][0])


1633

0.003517718120415765

[0.003517718120415765, -0.017199529215374695]

array([271.79450322])

array([[0.00351772],
       [0.0219465 ],
       [0.051092  ],
       ...,
       [0.3131765 ],
       [0.31382569],
       [0.26383715]])

In [242]:
# (NEW)

for i in range(len(frag_type)):
    # Pulling local variables into new, temp variables
    output = locals()["sortOut_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(output)): # Number of buckets (i.e. 16 or 8)
        nFrags = output[j][0].shape[0]
        temp2 = []
        for k in range(0, nFrags, 1): # Number of fragments #output[0][0].shape[0]
            pos_X = float(output[j][0][k])
            pos_Y = float(output[j][1][k])
            vel_X = float(output[j][2][k])
            vel_Y = float(output[j][3][k])
            acc_X = float(output[j][4][k])
            acc_Y = float(output[j][5][k])
            temp = [pos_X, pos_Y, vel_X, vel_Y, acc_X, acc_Y]
            temp2.append(np.array(temp))
            #locals()[decoder_output][j][k].append(np.array(temp))
        #locals()[decoder_output][j].append(np.array(temp2))
        locals()[decoder_output].append(np.array(temp2))
        #temp = np.array(np.concatenate((vel_X, vel_Y, acc_X, acc_Y, outputX[j], outputY[j]),axis=1))


In [245]:
decoder_output_AD
decoder_output_AD[0].shape
decoder_output_AD[0][0]

[array([[ 3.51771812e-03, -1.71995292e-02, -2.49460149e-04,
         -2.65589862e-04,  2.81711322e+01,  2.71794503e+02],
        [ 2.19465007e-02, -5.74727803e-03,  1.39547467e-03,
          1.10366072e-03,  2.84099939e+01,  2.71442115e+02],
        [ 5.10919955e-02,  2.47684665e-02,  6.49772513e-04,
          1.04829101e-03,  2.93819730e+01,  2.71698756e+02],
        ...,
        [ 3.13176503e-01,  1.09683733e-01,  4.74592322e-03,
         -7.98282391e-03,  9.31909748e+00,  2.50593419e+02],
        [ 3.13825687e-01, -1.74191717e-01, -2.96011980e-03,
         -1.22261677e-02,  1.75705132e+01,  2.49847558e+02],
        [ 2.63837147e-01, -4.04205834e-01, -5.11648224e-04,
         -5.13681521e-03,  2.46097445e+01,  2.42113592e+02]]),
 array([[4.79588285e-03, 7.44815703e-02, 3.06038591e-03, 3.17782722e-03,
         2.08739799e+01, 1.84401652e+02],
        [5.96350295e-02, 1.53963955e-01, 1.37906432e-03, 3.10392178e-03,
         2.18018433e+01, 1.87302336e+02],
        [1.00848788e-01, 2.38

(1633, 6)

array([ 3.51771812e-03, -1.71995292e-02, -2.49460149e-04, -2.65589862e-04,
        2.81711322e+01,  2.71794503e+02])

### Part B: Preprocessing Decoder Data

In [246]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
import sys

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.metrics import get_R2_parts

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

In [221]:
# (OLD)
#For the Kalman filter, we use the position, velocity, and acceleration as outputs.
#Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    
    # Pulling local variables into new, temp variables
    outputX = locals()["sortX_"+frag_type[i]]
    outputY = locals()["sortY_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(outputX)):
        # Determine X and Y velocities 
        vel_X = (np.diff(outputX[j], axis = 0))      
        vel_X = np.concatenate((vel_X,vel_X[-1:,:]),axis=0)
        vel_Y = (np.diff(outputY[j], axis = 0))      
        vel_Y = np.concatenate((vel_Y,vel_Y[-1:,:]),axis=0)

        # Determine X and Y accelerations 
        acc_X = (np.diff(vel_X, axis = 0))   
        acc_X = np.concatenate((acc_X,acc_X[-1:,:]),axis=0)
        acc_Y = (np.diff(vel_Y, axis = 0))      
        acc_Y = np.concatenate((acc_Y,acc_Y[-1:,:]),axis=0)

        # Concatenating all 3 kinematics for X and Y components
        # Corrected decoder outputs (format = x,y vel, x,y acc, x,y pos)
        temp = np.array(np.concatenate((vel_X, vel_Y, acc_X, acc_Y, outputX[j], outputY[j]),axis=1))

        locals()[decoder_output].append(temp)

In [227]:
decoder_output_AD[0].shape
decoder_output_AD[0][-1].shape
decoder_output_AD[0][-1]
len(decoder_output_AD)
decoder_output_AD

(1633, 6)

(6,)

array([  7.03923131,  -7.73396546,   0.        ,   0.        ,
        24.60974452, 242.11359228])

16

[array([[ 2.38861708e-01, -3.52387935e-01,  7.33117368e-01,
          6.09028943e-01,  2.81711322e+01,  2.71794503e+02],
        [ 9.71979076e-01,  2.56641007e-01, -2.39278454e+01,
          6.05714906e+00,  2.84099939e+01,  2.71442115e+02],
        [-2.29558664e+01,  6.31379007e+00,  2.61503313e+01,
         -6.65196604e+00,  2.93819730e+01,  2.71698756e+02],
        ...,
        [ 8.25141574e+00, -7.45860819e-01, -1.21218443e+00,
         -6.98810465e+00,  9.31909748e+00,  2.50593419e+02],
        [ 7.03923131e+00, -7.73396546e+00,  0.00000000e+00,
          0.00000000e+00,  1.75705132e+01,  2.49847558e+02],
        [ 7.03923131e+00, -7.73396546e+00,  0.00000000e+00,
          0.00000000e+00,  2.46097445e+01,  2.42113592e+02]]),
 array([[  0.92786338,   2.90068393,   1.04446409,   1.99894005,
          20.87397992, 184.40165161],
        [  1.97232747,   4.89962398,   1.49847817,   2.53531146,
          21.8018433 , 187.30233554],
        [  3.47080564,   7.43493545,   2.49662926,   

### Part C: Partitioning and Running the Kalman Filter on the Same Buckets 

In [247]:
#Set what part of data should be part of the training/testing/validation sets

training_range=[0, 0.8]
valid_range=[0.8,0.9]
testing_range=[0.9, 1]

In [248]:
# Running decoder on X and Y components together to get the prediction nom and denom to later compute the combined XY_FVAF
from Neural_Decoding.runModelsKF import run_model_kf

for i in range(len(frag_type)):

    # Pulling local variables 
    input = locals()["sortIn_"+frag_type[i]]
    output = locals()["decoder_output_"+frag_type[i]]

    # Creating a local variable to hold (X, Y) predicted outputs for each frag type 
    parts = "pred_parts_" + frag_type[i]
    locals()[parts] = run_model_kf(input, output, training_range, testing_range, valid_range, "parts")

/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:48: RuntimeWarning: invalid value encountered in true_divide
  X_train=(X_train-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:49: RuntimeWarning: divide by zero encountered in true_divide
  X_test=(X_test-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:49: RuntimeWarning: invalid value encountered in true_divide
  X_test=(X_test-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:50: RuntimeWarning: invalid value encountered in true_divide
  X_valid=(X_valid-X_train_mean)/X_train_std


In [249]:
pred_parts_AD[0][0][5]

86177.52041784716

In [250]:
# Compute combined XY_FVAF (velocity only)
from Neural_Decoding.metrics import compute_XY_FVAF

for i in range(len(frag_type)):

    # Pulling previously computed predicted_parts (i.e. nom and denom)
    parts = locals()[ "pred_parts_" + frag_type[i]]
    
    # Creating a local variable to hold XY_FVAFs each frag type 
    XY_FVAF = "XY_FVAF_" + frag_type[i]
    locals()[XY_FVAF] = []

    for j in range(len(parts)):
        #curr_bucket = Kalman_R2s_combined[i]
        vel_x_nom = parts[j][0][0] # dim = (curr_bucket, nom, x_vel)
        vel_x_denom = parts[j][1][0] # dim = (curr_bucket, denom, x_vel)
        vel_y_nom = parts[j][0][1] # dim = (curr_bucket, nom, y_vel)
        vel_y_denom = parts[j][1][1] # dim = (curr_bucket, denom, y_vel)

        curr_bucket_XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
        locals()[XY_FVAF].append(curr_bucket_XY_FVAF)

In [254]:
XY_FVAF_HV

[0.11312576051669809,
 nan,
 0.3009460206465555,
 0.17672436958159832,
 0.3103356178769259,
 0.2847933442835944,
 0.15725940333019095,
 0.20430512492659825,
 0.0555907420103362,
 0.08908951041409252,
 0.23587576205121497,
 0.12279699280365264,
 0.25135105868315133,
 0.14666016348559596,
 0.1346890712506058,
 0.1115590403380633]

### Part D: Partitioning and Running the Kalman Filter on Separate Training and Test Buckets